In [1]:
import numpy as np
import torch
from torch import nn
from torchvision import transforms
import cv2 as cv
from torch.utils.data import Dataset, DataLoader
from typing import *
import sys
import os
import random
from utils import Accumulator, train
from net import resnet18
import pandas as pd



runtime_path = sys.path[0]
runtime_path


'/home/wakinghours/programming/LiMu-DeepLearning/kaggle/cifar10'

In [2]:
def try_all_GPUS() -> List[torch.device]:
    devices = [torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    return devices if devices else torch.device("cpu")

devices = try_all_GPUS()


In [3]:
path_join = lambda *args: os.path.join(*args)


class CIFAR10_dataset(Dataset):
    def __init__(self, type_dataset: str = "train", vaild_rate=0.1) -> None:
        super().__init__()
        self.type_dataset = type_dataset
        self.vaild_rate = vaild_rate
        self.transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(42),
            transforms.RandomResizedCrop(32, (0.6, 1.0), ratio=(1.0, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.Normalize([0.4914, 0.4822, 0.4465],  # normalize. 归一化.
                                 [0.2023, 0.1994, 0.2010])
        ])
        self.transform_vaild = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize([0.4914, 0.4822, 0.4465],  # normalize. 归一化.
                                         [0.2023, 0.1994, 0.2010])
                ])
        self.labels_dict = self.parse_csv2label()
        self.classes = ['airplane', 'automobile', 'bird', 'cat',
                        'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

        self.root_path = path_join(runtime_path, "train")
        if "train" == self.type_dataset:
            # generate train and vaild dataset file.
            self.shuffle_train_vaild()
            # only generate in "train" mode.

            with open("./train.txt", "r") as f:
                self.file_path_list = f.readlines()

        elif "vaild" == self.type_dataset:
            with open("./train_vaild.txt", "r") as f:
                self.file_path_list = f.readlines()
        else: # test:
            self.root_path = path_join(runtime_path, "test")

            self.file_path_list = [path_join(self.root_path, file_name) for file_name in os.listdir(self.root_path)]


    def __getitem__(self, index):
        file_path = self.file_path_list[index].strip()
        file_name = file_path.split("/")[-1].split(".")[0]
        img = cv.imread(file_path)
        if "train" == self.type_dataset:
            X = self.transform_train(img)
            return X, self.classes.index(self.labels_dict[file_name])
        elif "vaild" == self.type_dataset:
            X = self.transform_vaild(img)

            return X, self.classes.index(self.labels_dict[file_name])

        else: # test
            X = self.transform_vaild(img)
            return X, file_name

        


    def parse_csv2label(self):
        with open("./trainLabels.csv", "r") as f:
            return {ele[0]: ele[1] for ele in [line.strip().split(',') for line in f.readlines()][1:]}

    def shuffle_train_vaild(self):
        l = len(os.listdir(self.root_path))

        try:
            file_name_list = os.listdir(self.root_path)
            random.shuffle(file_name_list)

            with open(path_join(self.root_path, "../", "train.txt"), "w") as train_file_writer:
                train_file_writer.write(
                    "\n".join([path_join(self.root_path,  file_name)
                              for file_name in file_name_list[0: int(l*(1-self.vaild_rate))]])
                )

            with open(path_join(self.root_path, "../",  "train_vaild.txt"), "w") as train_file_writer:
                train_file_writer.write(
                    "\n".join([path_join(self.root_path,  file_name)
                              for file_name in file_name_list[int(l*(1-self.vaild_rate)):]])
                )

        except Exception as e:
            print("error: ", e)

    def __len__(self):
        return len(self.file_path_list)




def load_CIFAR10_iter(batch_size=64, num_workers=28):
    return DataLoader(
        CIFAR10_dataset("train"),
        batch_size,
        shuffle=True,
        num_workers=num_workers
    ),     DataLoader(
        CIFAR10_dataset("vaild"),
        batch_size,
        shuffle=True,
        num_workers=num_workers
    )




In [4]:
# net = resnet18() # 实例化. 
import torchvision
net = torchvision.models.resnet34(pretrained=True)

# net = torchvision.models.resnet101(pretrained=True)

epoch = 150
batch_size = 512
lr = 5e-3
loss_fn = torch.nn.CrossEntropyLoss()

/home/wakinghours/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wakinghours/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
net.fc = nn.Linear(512, 10)
print(net)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:

train_iter, vaild_iter = load_CIFAR10_iter(batch_size)

train(
    net,
    loss_fn,
    train_iter, vaild_iter, 
    lr, epoch, 
    80, 10, 0.9, 5e-3,
    "logs/epoch150_testacc0.78_loss0.49_acc0.83.pth"
)



train on:  [device(type='cuda', index=0), device(type='cuda', index=1)]
10 test acc: 0.6993303537368775 train loss: 0.6332733068953861 train acc: 0.7847734906456687
20 test acc: 0.7006815969944 train loss: 0.6478056887334044 train acc: 0.7805868874896656
30 test acc: 0.6643734037876129 train loss: 0.6541716199029576 train acc: 0.7791890170086514
40 test acc: 0.6612324595451355 train loss: 0.6507146839391101 train acc: 0.7795768386938355
50 test acc: 0.7230070114135743 train loss: 0.6416514780033719 train acc: 0.7811024256727912
60 test acc: 0.7278698980808258 train loss: 0.6492209190672095 train acc: 0.7787618684497747
70 test acc: 0.7238600134849549 train loss: 0.6678034101020206 train acc: 0.7724936455488205
80 test acc: 0.741497927904129 train loss: 0.6548780284144662 train acc: 0.7779091285033659
90 test acc: 0.7174306452274323 train loss: 0.6594853814352643 train acc: 0.774488823657686
100 test acc: 0.7041374325752259 train loss: 0.642893907698718 train acc: 0.7807944267988205
110

In [7]:
test_datasets = CIFAR10_dataset("test")
test_iter = DataLoader(
    test_datasets,
    100000,
    shuffle=False,
    num_workers=30,
)

classes = test_datasets.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [8]:
def evel_test_iter(net, test_iter: DataLoader, load_path = None, devices=try_all_GPUS()):
    net = torch.nn.DataParallel(net, devices).to(device=devices[0])
    pred, id = [], []

    net.eval()
    with torch.no_grad():
        # result = [(classes[net(features).argmax(dim=1).cpu().numpy()[0]], int(file_name[0])) for features, file_name in test_iter]
        for features, file_name in test_iter:
            pred.extend(list(net(features).argmax(dim=1).cpu().numpy()))
            id.extend([int(ele) for ele in file_name])

        # print(result[0])
    
    result = zip(id, pred)
    retult = sorted(result, key=lambda x: x[0])
    df = pd.DataFrame({'id': [tup[0] for tup in retult], 'label':  [tup[1] for tup in retult]})
    df['label'] = df['label'].apply(lambda x: classes[x])
    df.to_csv('./submission.csv', index=False)
    

evel_test_iter(net, test_iter)
